In [1]:
import scipy.io as scio
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt

pixel_savepath = 'ICVL-pixel数据/'
pixel_data = scio.loadmat(pixel_savepath+'1by19 162_train_icvl pixel.mat')['data']

#pixel_data[pixel_data==0] = 1 # 防除0
#print(pixel_data.shape)
#print(pixel_data.max(),pixel_data.min())
#(14183255, 31)
#4095.0 0.0625
#plt.hist(pixel_data)
K=200 # e-/DN
tc=10**-3 # s 1ms
h_p=6.62607015*10**-17 # *10**-17J/s
c_light = 3 # 10**8m/s
wavelength_m = np.arange(400,710,10) # 10**-9m
light_visual_F = np.zeros([wavelength_m.shape[0],2])
light_visual_F[:,0]=wavelength_m
pixel_S = 6*6*10**-12# m**
temp = scio.loadmat('data/Camera spectral response function/light visual function.mat')['data'][:,1]

light_visual_F[:,1]=temp[4:65:2]
font1 = {'family' : 'Times New Roman',
'weight' : 'normal',
'size'   : 16,
}
plt.figure(figsize=(5,5))
plt.title('light visual function',font1)
plt.xlabel('lambda/nm',font1)
plt.plot(light_visual_F[:,0],light_visual_F[:,1],'-*')
plt.legend(['V'])
#plt.savefig('data/Camera spectral response function/' + 'light visual function.png')
plt.show()
Kmax= 683 #lm/W

# 1.4MCCD量子效率三次样条插值
QE_data = scio.loadmat('data/Camera spectral response function/1.4MCCDQE_splrep.mat')['filters']
print(QE_data.shape)
font1 = {'family' : 'Times New Roman',
'weight' : 'normal',
'size'   : 16,
}
plt.figure(figsize=(5,5))
plt.title('1.4MCCDQE_splrep',font1)
plt.xlabel('lambda/nm',font1)
plt.plot(np.arange(400,710,10),QE_data)
plt.legend(['QE/%'])
#plt.savefig(img_path_data + '1.4MCCDQE_splrep.png')
plt.show()

<Figure size 500x500 with 1 Axes>

(31, 1)


<Figure size 500x500 with 1 Axes>

In [2]:
spectum = np.ones([1,31])*2**12*K*h_p*c_light/tc*Kmax/pixel_S/QE_data.reshape(-1)/0.01/wavelength_m*light_visual_F[:,1] # 太阳有10wlux
#                       DN *放大系数 /量子效率=光子数 * 单光子能量 = 焦耳 /曝光时间tc=光辐射功率 /单位像素面积=光辐射照度  * Kmax*明视觉函数=光照度
print(spectum)#
print(sum(sum(spectum)))


[[7.18483791e+00 1.96402582e+01 5.93477831e+01 1.58989946e+02
  2.94114384e+02 4.57700786e+02 6.87075035e+02 9.99756777e+02
  1.47992983e+03 2.16659916e+03 3.32634046e+03 5.17814742e+03
  7.37448166e+03 9.09690836e+03 1.02762223e+04 1.09585684e+04
  1.11968712e+04 1.09362960e+04 1.02017590e+04 9.06817054e+03
  7.73596060e+03 6.32896771e+03 4.93514500e+03 3.54475683e+03
  2.42509601e+03 1.54113936e+03 9.16226409e+02 5.02938983e+02
  2.80552728e+02 1.42771347e+02 7.53980312e+01]]
122373.0570902289


In [3]:
# 单位像素的照度=DN*K/QE  *  h*c/λ  /t /像素面积
QE_data = QE_data.reshape(-1,31)[-1]/100
print(wavelength_m.shape,QE_data.shape)
print(QE_data)
print(wavelength_m)
all_filter = QE_data*wavelength_m
print(all_filter)

(31,) (31,)
[0.43     0.464237 0.495783 0.52421  0.549091 0.57     0.586509 0.59819
 0.604617 0.605363 0.6      0.588449 0.572017 0.552361 0.531137 0.51
 0.490257 0.471821 0.454257 0.437129 0.42     0.402523 0.384697 0.36661
 0.348349 0.33     0.311651 0.29339  0.275303 0.257477 0.24    ]
[400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570
 580 590 600 610 620 630 640 650 660 670 680 690 700]
[172.      190.33717 208.22886 225.4103  241.60004 256.5     269.79414
 281.1493  290.21616 296.62787 300.      300.10899 297.44884 292.75133
 286.81398 280.5     274.54392 268.93797 263.46906 257.90611 252.
 245.53903 238.51214 230.9643  222.94336 214.5     205.68966 196.5713
 187.20604 177.65913 168.     ]


In [4]:
pixel_W = pixel_data/all_filter*K*h_p*c_light/tc/pixel_S # 瓦特每平方米

In [5]:
#print(K*2**12/0.2*h_p*c_light/500/tc/pixel_S) # lm/平方米/波长
print(pixel_W.shape,pixel_W.max(),pixel_W.min())

#print(pixel_lm.shape,pixel_lm.max(),pixel_lm.min())

(15131836, 31) 26.918409984374996 0.6676685963645834


In [6]:
#print(QE_data,'\n',pixel_data[0:2,:],'\n',pixel_data[0:2,:]/QE_data)
#print(pixel_lm.shape,pixel_lm.max(),pixel_lm.min())
scio.savemat(pixel_savepath+'no_selected_pixel_W_onlyICVL.mat',{'data':pixel_W})